Link to datasets: https://drive.google.com/drive/folders/149NUN6AWG6I-CINW_T1HNn-BBxNPqLVW?usp=sharing

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

/Users/skp340/opt/anaconda3/envs/ndmd-env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datetime import datetime
from functools import reduce
from functools import partial
import operator
import os
from timeit import default_timer

import einops
import h5py
import matplotlib.pyplot as plt
import numpy as np
import scipy.io

%matplotlib notebook

In [3]:
print(torch.cuda.is_available())

torch.manual_seed(0)
np.random.seed(0)

False


## Utilities

In [4]:
class LpLoss(object):
    """Computes relative and absolute L^{p} loss functions."""

    def __init__(self, d=2, p=2, size_average=True, reduction=True):
        super(LpLoss, self).__init__()

        assert d > 0 and p > 0

        self.dim = d
        self.p = p
        self.reduction = reduction
        self.size_average = size_average

    def abs(self, x, y):
        """Computes absolute error, assuming a uniform mesh."""

        batch_size = x.shape[0]

        h = 1.0 / (x.shape[1] - 1.0)

        all_norms = (h**(self.dim/self.p))*torch.norm(
                            x.view(batch_size,-1) - y.view(batch_size,-1),
                            self.p,
                            1,) # axis

        if self.reduction:
            if self.size_average:
                return torch.mean(all_norms)
            else:
                return torch.sum(all_norms)

        return all_norms

    def rel(self, x, y):
        """Computes relative error."""

        num_examples = x.size()[0]

        diff_norms = torch.norm(
                        x.reshape(num_examples,-1) - y.reshape(num_examples,-1),
                        self.p,
                        1,) # axis
        y_norms = torch.norm(y.reshape(num_examples,-1),
                            self.p,
                            1,)

        if self.reduction:
            if self.size_average:
                return torch.mean(diff_norms/y_norms)
            else:
                return torch.sum(diff_norms/y_norms)

        return diff_norms/y_norms

    def __call__(self, x, y):
        return self.rel(x, y)


In [5]:
class MatReader(object):
    def __init__(self, file_path, to_torch=True, to_cuda=False, to_float=True):
        super(MatReader, self).__init__()

        self.to_torch = to_torch
        self.to_cuda = to_cuda
        self.to_float = to_float

        self.file_path = file_path

        self.data = None
        self.old_mat = None
        self._load_file()

    def _load_file(self):
        try:
            self.data = scipy.io.loadmat(self.file_path)
            self.old_mat = True
        except:
            self.data = h5py.File(self.file_path)
            self.old_mat = False

    def load_file(self, file_path):
        self.file_path = file_path
        self._load_file()

    def read_field(self, field):
        x = self.data[field]

        if not self.old_mat:
            x = x[()]
            x = np.transpose(x, axes=range(len(x.shape) - 1, -1, -1))

        if self.to_float:
            x = x.astype(np.float32)

        if self.to_torch:
            x = torch.from_numpy(x)

            if self.to_cuda:
                x = x.cuda()

        return x

    def set_cuda(self, to_cuda):
        self.to_cuda = to_cuda

    def set_torch(self, to_torch):
        self.to_torch = to_torch

    def set_float(self, to_float):
        self.to_float = to_float

In [6]:
class UnitGaussianNormalizer(object):
    """Applies pointwise Gaussian normalization to an input tensor."""
    def __init__(self, x, eps=0.00001):
        super(UnitGaussianNormalizer, self).__init__()
        """Args:
            x: tensor of shape (ntrain X n) or (ntrain X T X n) or
                (ntrain X n X T)
        """
        self.mean = torch.mean(x, 0)
        self.std = torch.std(x, 0)
        self.eps = eps

    def encode(self, x):
        x = (x - self.mean) / (self.std + self.eps)
        return x

    def decode(self, x, sample_idx=None):
        if sample_idx is None:
            std = self.std + self.eps # n
            mean = self.mean
        else:
            if len(self.mean.shape) == len(sample_idx[0].shape):
                std = self.std[sample_idx] + self.eps  # (batch X n)
                mean = self.mean[sample_idx]
            if len(self.mean.shape) > len(sample_idx[0].shape):
                std = self.std[:,sample_idx]+ self.eps # (T X batch X n)
                mean = self.mean[:,sample_idx]

        # x in shape of (batch X n) or (T X batch X n)
        x = (x * std) + mean
        return x

    def cuda(self):
        self.mean = self.mean.cuda()
        self.std = self.std.cuda()

    def cpu(self):
        self.mean = self.mean.cpu()
        self.std = self.std.cpu()

    def to(self, device):
        self.mean = self.mean.to(device)
        self.std = self.std.to(device)

In [7]:
def mat_to_tensor1d(TRAIN_PATH, 
                  TEST_PATH, 
                  ss_rate, 
                  x_field, y_field, 
                  vsamples=None, 
                  normalize=False):
    """Converts .mat file contents to torch tensors.

        Args:
            TRAIN_PATH: list of .mat file names to concatenate into training 
                        tensors
            TEST_PATH: list of .mat file names to concatenate into test tensors
            ss_rate: [train subsampling rate, test rate]
            x_field, y_field: names of fields in the .mat file
            vsamples: [ntest, ntrain]
            print: Print log contents to the console; default = True
            normalize: Apply normalization to the tensors; default = False
    """
    t1 = default_timer()

    reader = MatReader(TRAIN_PATH)
    x_data = reader.read_field(x_field)
    y_data = reader.read_field(y_field)
    dimension = len(x_data.shape) - 1
    
    mat_info = ("input signal vector samples: {}\n"
                   "output signal vector samples: {}\n"
                   "input signal entry samples: {}\n"
                   "output signal entry samples: {}\n"
                   "signal dimension: {}\n\n"
                  ).format(x_data.shape[0], 
                           y_data.shape[0], 
                           x_data.shape[1], 
                           y_data.shape[1], 
                           dimension)

    ntrain = vsamples[0]
    ntest = vsamples[1]

    full_res = x_data.shape[1]
    tr_ss = ss_rate[0]
    tst_ss = ss_rate[1]
    tr_esamples = int(((full_res - 1)/tr_ss) + 1)

    x_train = x_data[ntrain:,::tr_ss][:,:tr_esamples]
    y_train = y_data[ntrain:,::tr_ss][:,:tr_esamples]

    if TRAIN_PATH != TEST_PATH:
        # using separate test/train datasets
        test_reader = MatReader(TEST_PATH)
        x_test = test_reader.read_field(x_field)
        y_test = test_reader.read_field(y_field)

        full_res = x_test.shape[1]
        tst_esamples = int(((full_res - 1)/tst_ss) + 1)

        x_test = x_test[ntest:,::tst_ss][:,:tst_esamples]
        y_test = y_test[ntest:,::tst_ss][:,:tst_esamples]

    else:
        full_res = x_data.shape[1]
        tst_esamples = int(((full_res - 1)/tst_ss) + 1)

        # same dataset; use last (ntest) samples
        x_test = x_data[-ntest:,::tst_ss][:,:tst_esamples]
        y_test = y_data[-ntest:,::tst_ss][:,:tst_esamples]

        
    ds_info = ("training dataset: {}\n"
                    "test dataset: {}\n\n"
                    "input train samples: {}\n"
                    "output train samples: {}\n"
                    "input train resolution: {}\n"
                    "output train resolution: {}\n\n"
                    "input test samples: {}\n"
                    "output test samples: {}\n"
                    "input test resolution: {}\n"
                    "output test resolution: {}\n\n"
    ).format(TRAIN_PATH,
             TEST_PATH,
             x_train.shape[0], 
             y_train.shape[0], 
             x_train.shape[1],
             y_train.shape[1],
             x_test.shape[0],
             y_test.shape[0],
             x_test.shape[1],
             y_test.shape[1])


    t2 = default_timer()
    
    return x_train, y_train, x_test, y_test, mat_info, ds_info

## Kernel

In [8]:
def complex_mul1d(a, b):
    """Performs complex multiplication between two input tensors.

    Args:
    a: tensor of shape (batch, in_channel, x)
    b: tensor of shape (in_channel, out_channel, x)

    return: tensor of form (batch, out_channel, x)
    """
    return torch.einsum("bix,iox->box", a, b)

In [ ]:
class SpectralConv1d(nn.Module):
    def __init__(self, 
                 in_channels, 
                 out_channels, 
                 modes, 
                 trainable=True):
        super(SpectralConv1d, self).__init__()

        """Applies FFT, linear transform, and inverse FFT.
        
        Args:
            in_channels: number of input channels
            out_channels: number of output channels
            modes: number of Fourier modes to multiply
            train: make weights trainable parameters
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes = modes
        self.trainable = trainable
        self.scale = (1 / (in_channels*out_channels))

        self.weights = nn.Parameter(self.scale *
                                    torch.rand(self.in_channels, 
                                               self.out_channels, 
                                                self.modes, 
                                                dtype=torch.cfloat))
        self.weights.requires_grad = trainable

    def forward(self, x):
        batchsize = x.shape[0]
        
        # Compute Fourier coeffcients up to factor of e^(-constant)
        x_ft = torch.fft.rfft(x)

        # Multiply relevant Fourier modes
        out_ft = complex_mul1d(x_ft[:, :, :self.modes], 
                               self.weights)

        # Return to physical space
        x = torch.fft.irfft(out_ft,n=x.size(-1))
        
        return x

In [ ]:
class FourierBlock1d(nn.Module):
    """Fourier kernel block implementing
                    u' = (W + K)(u),
       where W is defined by a convolution and K is defined by 
       a spectral convolution in Fourier space.
    """
    def __init__(self,
                 width,
                 modes,
                 nonlinearity,
                 trainable=True):
        super(FourierBlock1d, self).__init__()
        """
        Args:
            modes: Number of Fourier modes to multiply (at most floor(N/2) + 1)
            width: Dimension of lifting transform performed by first layer
            nonlinearity: Component-wise nonlinear activation
            trainable: Whether or not the weights are trainable parameters; 
                   default = True
        """
        self.width = width
        self.modes = modes
        self.trainable = trainable
        self.nonlinearity = nonlinearity
        
        # call signature SpectralConv2d(in_dim, out_dim, modes_x)
        self.R = SpectralConv1d(self.width,
                                self.width,
                                self.modes,
                                self.trainable)
        
        # call signature ConvNd(in_dim, out_dim, kernel_size)
        self.w = nn.Conv1d(self.width, self.width, 1) # TODO: implement with linear layer instead
        self.w.weight.requires_grad = trainable
        self.w.bias.requires_grad = trainable
        
    def forward(self, x):

        # x = 'b c x'
        Rx = self.R(x)

        # x = 'b c x'
        wx = self.w(x)

        if self.nonlinearity == None:
            return Rx + wx
        else:
            return self.nonlinearity(Rx + wx)

In [ ]:
class FourierKernel1d(nn.Module):
    """The Fourier kernel containing a specified number of Fourier blocks.
    
    1. Lift the input to the desired channel dimension by self.fc0.
    2. Apply specified number of iterations of the operator 
                            u' = (W + K)(u)
    3. Project from the channel space to the output space by self.fc1 
        and self.fc2.
    """
    def __init__(self, 
                 n_blocks, 
                 modes, 
                 width,
                 nonlinearity,
                 trainable):
        """
        Args:
            blocks: Number of kernel integral transform blocks
            modes: Number of Fourier modes to multiply (at most floor(N/2) + 1)
            width: Dimension of lifting transform performed by first layer
            nonlinearity: Component-wise nonlinear activation
        """
        super(FourierKernel1d, self).__init__()

        self.n_blocks = n_blocks
        self.modes = modes
        self.width = width
        self.nonlin = nonlinearity
        self.trainable = trainable

        # input channel is 2: (a(x), x)
        self.fc0 = nn.Linear(2, self.width)
        
        self.blocks = nn.ModuleList()
        
        for b in range(self.n_blocks):
            
            if b == self.n_blocks - 1:
                self.blocks.append(FourierBlock1d(
                                        self.width,
                                        self.modes,
                                        None,
                                        trainable))
            else:
                self.blocks.append(FourierBlock1d(
                                            self.width,
                                            self.modes,
                                            self.nonlin,
                                            trainable))
        
        # return to input space
        self.fc1 = nn.Linear(self.width, self.width*self.n_blocks)
        self.fc2 = nn.Linear(self.width*self.n_blocks, 1)

        
    def forward(self, x):
        """Apply kernel transform to input data.
        Args:
            x: the solution of the coefficient function and locations 
                    (a(x), x); shape (batchsize, x=s, c=2)
            
            return: solution u(x); shape (batchsize, x=s, c=1)
        """
        batchsize = x.shape[0]
        
        x = self.fc0(x)
        
        x = einops.rearrange(x, 'b x c -> b c x')

        for block in self.blocks:
            x = block(x)

        x = einops.rearrange(x, 'b c x -> b x c')
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        
        return x

In [ ]:
class OperatorNet(nn.Module):    
    def __init__(self, 
                 kernel, 
                 name=None):
        super(OperatorNet, self).__init__()
        
        self.kernel = kernel
        self.modelname = name
        
        c = 0
        for p in self.parameters():
            c += reduce(operator.mul, list(p.size()))
        self.param_count = c
        
    def param_count(self):
        return self.param_count
    
    def name(self):
        return "model: {}".format(self.modelname)
        
    def forward(self, x):
        x = self.kernel(x)
        return x.squeeze()

# Training

In [ ]:
PATH = "../data/"

### Set up properties and I/O of training run

In [ ]:
"""
train_ds_name = "burgers_e4096_N1000_g2.5_t7_v100_s49_per_2104291732"
test_dir = "burgers_e4096_N400_g2.5_t7_v100_s49_dir_2104300546"
test_per = "burgers_e4096_N400_g2.5_t7_v100_s49_per_2104300508"
test_neu = "burgers_e4096_N400_g2.5_t7_v100_s49_neu_2104300435"

TEST_PATH = [PATH+test_dir+'.mat',
             PATH+test_per+'.mat',
             PATH+test_neu+'.mat']
"""

train_ds_name = 'burgers_data_R10_original'
test_ds_name = "burgers_data_R10_original"
test_ds_name = 'burgers_e4096_N400_g2.5_t7_v100_s49_per_2104300508'
TRAIN_PATH = PATH+train_ds_name+'.mat'
TEST_PATH = PATH+test_ds_name+'.mat'

# field name in the matlab file
x_name = "a"
y_name = "u"

# dataset properties
ss_rate = [8, 8]
split = 0.8
vsamples = [500, 200]

In [ ]:
# model properties
n_blocks = 4
modes = 16
width = 256

# training properties
trainable_blocks = True
learning_rate = 0.001
epochs = 250
step_size = 100
gamma = 0.5
batch_size = 20
nonlinearity = F.relu
# I/O properties
output_freq = 1
checkpoint_freq = 1000

now = datetime.now()
ts = now.strftime("%m%d%Y%H%M%S")

model_name = "{}_{}_{}_{}_{}_{}_{}".format(n_blocks,
                                           train_ds_name, 
                                           ss_rate[0], 
                                           vsamples[0], 
                                           modes, 
                                           width, 
                                           ts)
model_path = PATH+"model/{}".format(model_name)
train_err_path = PATH+"results/{}_train.txt".format(model_name)
test_err_path = PATH+"results/{}_test.txt".format(model_name)
fig_path = PATH+"figures/{}_plot.png".format(model_name)

checkpt = PATH+"log/{}.pt".format(model_name)
path_log = PATH+"log/{}.txt".format(model_name)


run_properties = ("run properties:\n"
                "num. epochs: {}\n"
                "learning rate: {}\n"
                "step size: {}\n"
                "gamma: {}\n"
                "n_blocks: {}\n"
                "training dataset: {}\n"
                "subsampling rate: {}\n"
                "training samples: {}\n"
                "modes: {}\n"
                "width: {}\n\n").format(epochs, 
                                        learning_rate, 
                                        step_size, 
                                        gamma, 
                                        n_blocks, 
                                        train_ds_name, 
                                        ss_rate[0], 
                                        vsamples[0], 
                                        modes, 
                                        width,
                                        )
print(run_properties)

train_log = []
train_log.append(run_properties)

In [ ]:
x_train, y_train, x_test, y_test, mat_info, ds_info = mat_to_tensor1d(
                                                            TRAIN_PATH, 
                                                            TEST_PATH, 
                                                            ss_rate,
                                                            x_name, 
                                                            y_name,
                                                            vsamples=vsamples,
                                                            )
print(mat_info)
print(ds_info)
train_log.append(mat_info)
train_log.append(ds_info)

### Add spatial coordinate channel

In [ ]:
ntrain = x_train.shape[0]
ntest = x_test.shape[0]
train_esamples = x_train.shape[1]
test_esamples = x_test.shape[1]

tr_domain = np.linspace(0, 2*np.pi, train_esamples).reshape(1, train_esamples, 1)
tr_domain = torch.tensor(tr_domain, dtype=torch.float)

x_train = torch.cat([x_train.view(ntrain,
                                  train_esamples,
                                  1), 
                     tr_domain.repeat(ntrain,1,1)],
                     dim=2)

tst_domain = np.linspace(0, 2*np.pi, test_esamples).reshape(1, test_esamples, 1)
tst_domain = torch.tensor(tst_domain, dtype=torch.float)
x_test = torch.cat([x_test.view(ntest,
                                test_esamples,
                                1), 
                    tst_domain.repeat(ntest,1,1)],
                    dim=2)

data_shape = ("x_train shape: {}\n"
             "x_test shape: {}\n"
             "y_train shape: {}\n"
             "y_test shape: {}\n\n"
             ).format(x_train.shape, 
                      x_test.shape, 
                      y_train.shape, 
                      y_test.shape)
print(data_shape)
train_log.append(data_shape)

### Make dataloaders from tensors

In [ ]:
ntrain = ntrain - (ntrain % batch_size)
ntest = ntest - (ntest % batch_size)

train_loader = torch.utils.data.DataLoader(
                            torch.utils.data.TensorDataset(
                                    x_train[:ntrain,:,:], 
                                    y_train[:ntrain,:]), 
                            batch_size=batch_size,
                            pin_memory=True,
                            shuffle=True)
test_loader = torch.utils.data.DataLoader(
                            torch.utils.data.TensorDataset(
                                    x_test[:ntest,:,:], 
                                    y_test[:ntest,:]), 
                            batch_size=batch_size, 
                            pin_memory=True,
                            shuffle=False)

train_features, train_labels = next(iter(train_loader))
batch_info = ("x batch shape: {}\n"
              "y batch shape: {}\n"
             ).format(train_features.size(), 
                      train_labels.size())
print(batch_info)
train_log.append(batch_info)

### Model, optimizer, loss

In [ ]:
device = torch.device('cpu')

if torch.cuda.is_available():
    device = torch.device('cuda')
    
gpu = "using device {}\n".format(device)
print(gpu)
train_log.append(gpu)


kernel = FourierKernel1d(n_blocks, 
                         modes, 
                         width, 
                         nonlinearity,
                         trainable_blocks).to(device)
model = OperatorNet(kernel,
                    name="Fourier{}blocks1d".format(n_blocks)).to(device)

print(model.name())
train_log.append(model.name())
print("model parameter count:", model.param_count)
train_log.append(("\nmodel parameter count: {}\n"
                ).format(model.param_count))

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                             lr=learning_rate, 
                             weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(
                                    optimizer, 
                                    step_size=step_size, 
                                    gamma=gamma)

lp_loss = LpLoss(size_average=False)
normalize = False
if normalize:
    y_normalizer.to(device)

RFM = False
# Random Features model uses average
if RFM:
    lp_loss = LpLoss(size_average=True)

### Training loop

In [ ]:
tr_err = []
tst_err = []

start_time = default_timer()
for ep in range(epochs):
        
    model.train()
    t1 = default_timer()
    train_mse = 0
    
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        
        if normalize:
            out = y_normalizer.decode(out)
            y = y_normalizer.decode(y)
        else:
            out = model(x)
        
        #loss = F.mse_loss(model(x).view(-1), y.view(-1), reduction='mean')
        loss = lp_loss(out.view(batch_size,-1), 
                       y.view(batch_size,-1))
        loss.backward()

        optimizer.step()
        train_mse += loss.item()

    scheduler.step()

    model.eval()
    abs_err = 0.0
    rel_err = 0.0
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            out = model(x)

            if normalize:
                out = y_normalizer.decode(model(x))

            abs_err += lp_loss.abs(out.view(batch_size,-1), 
                               y.view(batch_size,-1)).item()
            rel_err += lp_loss(out.view(batch_size,-1), 
                               y.view(batch_size,-1)).item()

    train_mse/= ntrain
    abs_err /= ntest
    rel_err /= ntest
    
    tr_err.append(train_mse)
    tst_err.append(rel_err)

    t2 = default_timer()
    
    epoch_vals = ("epoch {} \n"
                    "values:\n"
                    "elapsed time: {:.3f}\n"
                    "training mse: {:.5f}\n"
                    "absolute error: {:5f}\n"
                    "relative error: {:.5f}\n\n"
                 ).format(ep, t2-t1, train_mse, abs_err, rel_err)
    train_log.append(epoch_vals)
    
    if ep % output_freq == 0:
        print(epoch_vals)
        
    if ep % checkpoint_freq == 0:
        torch.save({
                    'epoch': ep,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'log': train_log,
                    }, checkpt)

end_time = default_timer()

In [ ]:
final_vals = ("final values:\n"
            "elapsed time: {:.3f}\n" 
            "training mse: {:.5f}\n"
            "relative error: {:.5f}\n\n"
             ).format(end_time - start_time, 
                      train_mse, 
                      rel_err)
print(final_vals)
train_log.append(final_vals)

fd = open(path_log, "a")
for string in train_log:
    fd.write(string)
fd.close()

torch.save(model, model_path)
np.savetxt(train_err_path, tr_err)
np.savetxt(test_err_path, tst_err)

# Visualize

In [ ]:
PATH = '../'

In [ ]:
name = "burgers_4blocks_samples"
data = []
dir = "../results/final/"+name
for filename in os.listdir(dir):
    f = os.path.join(dir, filename)
    if os.path.isfile(f):
        data.append(np.loadtxt(f))

labels = ['n = 400', 'n = 600', 'n = 800']
fig, ax = plt.subplots()
for arr, l in zip(data, labels):
    ax.plot(arr, label=l)

ax.set_xlabel('Epochs')
ax.set_ylabel('Relative Error')
ax.set_ylim(0, 0.10)
ax.legend()
fig.show()

fig.suptitle('Resolution: 4096, Blocks:4, Modes: 16, Width:64\nModel parameter count: 558,017')

save_path = "../figures/"+name+".png"
plt.savefig(save_path)

In [ ]:
burgers4 = np.loadtxt('../results/final/4_True_burgers_data_R10_original_4_600_16_64__05242021091147_test.txt')
burgers3 = np.loadtxt('../results/final/3_True_burgers_data_R10_original_2_600_16_64__05242021092125_test.txt')
burgers2 = np.loadtxt('../results/final/2_True_burgers_data_R10_original_2_600_16_64__05242021093013_test.txt')
burgers1 = np.loadtxt('../results/final/1_True_burgers_data_R10_original_2_600_16_64__05242021094828_test.txt')

arrays = [burgers4, burgers3, burgers2, burgers1]
labels = ['4 blocks (param. count: 558,017)', '3 blocks (param. count: 418,561)', '2 blocks (param. count: 279,105)', '1 block (param. count: 139,649)']
fig, ax = plt.subplots()
for arr, l in zip(arrays, labels):
    ax.plot(arr, label=l)
ax.set_xlabel("Epochs")
ax.set_ylabel("Relative Error")
ax.set_ylim(0, 0.06)
ax.legend()
fig.show()

fig.suptitle("Modes: 16, Width:64, Res. = 4096, Samples = 600")

im_path = "../figures/burgers_blocks.png"
plt.savefig(im_path)

In [ ]:
test800 = ('../results/_04212021135651fullres_darcy_e421_N1024_g2_t3_f1_2104130943_test.txt')
test800 = np.loadtxt(test800)
test600 = ('../results/_04212021140837fullres_darcy_e421_N1024_g2_t3_f1_2104130943_test.txt')
test600 = np.loadtxt(test600)
test400 = ('../results/_04212021141821fullres_darcy_e421_N1024_g2_t3_f1_2104130943_test.txt')
test400 = np.loadtxt(test400)
test200 = ('../results/_04212021143939fullres_darcy_e421_N1024_g2_t3_f1_2104130943_test.txt')
test200 = np.loadtxt(test200)

fig, ax = plt.subplots()
ax.plot(test200, label='n = 200')
ax.plot(test400, label='n = 400')
ax.plot(test600, label='n = 600')
ax.plot(test800, label='n = 800')
ax.set_xlabel('Epochs')
ax.set_ylabel('Relative Error')
ax.set_ylim(0, 0.10)
ax.legend()
fig.show()

fig.suptitle("Resolution: 211 (421//2), Blocks: 4, Modes: 12, Width: 32\nModel parameter count: 2,368,001")
im_path ="../figures/darcy200to800_ss_error_final.png"
plt.savefig(im_path)

In [ ]:
blocks4darcy = np.loadtxt(PATH+'results/_04212021062545fullres_darcy_e421_N1024_g2_t3_f1_2104130943_test.txt')
blocks3darcy = np.loadtxt(PATH+'results/_04212021071540fullres_darcy_e421_N1024_g2_t3_f1_2104130943_test.txt')
blocks2darcy = np.loadtxt(PATH+'results/_04212021075009fullres_darcy_e421_N1024_g2_t3_f1_2104130943_test.txt')

#blocks1darcy = np.loadtxt(PATH+'results/.txt')
fig, ax = plt.subplots()
ax.plot(blocks4darcy, label='blocks = 4, parameters = 2,368,001')
ax.plot(blocks3darcy, label='blocks = 3, parameters = 1,776,033')
ax.plot(blocks2darcy, label='blocks = 2, parameters = 1,184,065')
#ax.plot(blocks2darcy, label='blocks = 1, parameters = ')

ax.set_xlabel('Epochs')
ax.set_ylim(0, 0.10)
ax.legend()
fig.show()

fig.suptitle('Resolution: 421, Modes: 12, Width: 32, Samples: 820')
#fig.tight_layout()
im_path = PATH+"figures/darcyblocks_errorplot.png"
plt.savefig(im_path)

### Testing on different datasets:

In [ ]:
train_ds_name = "burgers_e4096_N1000_g2.5_t7_v100_s49_per_2104291732"
test_dir = "burgers_e4096_N400_g2.5_t7_v100_s49_dir_2104300546"
test_per = "burgers_e4096_N400_g2.5_t7_v100_s49_per_2104300508"
test_neu = "burgers_e4096_N400_g2.5_t7_v100_s49_neu_2104300435"

TEST_PATH = [PATH+'data/shared_data/'+test_dir+'.mat',
             PATH+'data/shared_data/'+test_per+'.mat',
             PATH+'data/shared_data/'+test_neu+'.mat']

train_ds_name = 'burgers_data_R10_original'
test_ds_name = 'burgers_data_R10_original'
TRAIN_PATH = PATH+'data/shared_data/'+train_ds_name+'.mat'
TEST_PATH = PATH+'data/shared_data/'+test_ds_name+'.mat'

# field name in the matlab file
x_name = "a"
y_name = "u"

# dataset properties
ss_rate = [2, 2]
split = 0.8
vsamples = [600, 200]

In [ ]:
x_train, y_train, x_test, y_test, mat_info, ds_info = mat_to_tensor1d(
                                                            TRAIN_PATH, 
                                                            TEST_PATH, 
                                                            ss_rate,
                                                            x_name, 
                                                            y_name,
                                                            vsamples=vsamples,
                                                            )
print(mat_info)
print(ds_info)

ntrain = x_train.shape[0]
ntest = x_test.shape[0]
train_esamples = x_train.shape[1]
test_esamples = x_test.shape[1]

tr_domain = np.linspace(0, 2*np.pi, train_esamples).reshape(1, train_esamples, 1)
tr_domain = torch.tensor(tr_domain, dtype=torch.float)

x_train = torch.cat([x_train.view(ntrain,
                                  train_esamples,
                                  1), 
                     tr_domain.repeat(ntrain,1,1)],
                     dim=2)

tst_domain = np.linspace(0, 2*np.pi, test_esamples).reshape(1, test_esamples, 1)
tst_domain = torch.tensor(tst_domain, dtype=torch.float)
x_test = torch.cat([x_test.view(ntest,
                                test_esamples,
                                1), 
                    tst_domain.repeat(ntest,1,1)],
                    dim=2)

data_shape = ("x_train shape: {}\n"
             "x_test shape: {}\n"
             "y_train shape: {}\n"
             "y_test shape: {}\n\n"
             ).format(x_train.shape, 
                      x_test.shape, 
                      y_train.shape, 
                      y_test.shape)
print(data_shape)

ntest = ntest - (ntest % batch_size)
test_loader = torch.utils.data.DataLoader(
                            torch.utils.data.TensorDataset(
                                    x_test[:ntest,:,:], 
                                    y_test[:ntest,:]), 
                            batch_size=batch_size, 
                            pin_memory=True,
                            shuffle=False)

In [ ]:
eval_device = 'cpu'
lp_loss = LpLoss(size_average=False)

#model = torch.load('../model/fullres_burgers_data_R10_04242021095651_04242021095651', 
 #                  map_location=torch.device(eval_device))
model = torch.load('../model/1_burgers_data_R10_2_800_16_256__04292021134321',map_location=torch.device('cpu'))
"""
training dataset: burgers_data_R10
modes: 16
width: 1024
layers: 4
train vsamples: 500
train esamples: 1024
viscosity: 1/10

x_train shape: torch.Size([500, 1024, 2])
x_test shape: torch.Size([200, 1024, 2])
y_train shape: torch.Size([500, 1024])
y_test shape: torch.Size([200, 1024])
model parameter count: 142,621,697
"""
model.eval()
rel_err = 0.0
with torch.no_grad():
    x, y = next(iter(test_loader))
    x, y = x.to(device), y.to(device)
    out = model(x)
    rel_err = lp_loss(out.view(-1), y.view(-1)).item()
    rel_err /= batch_size

In [ ]:
sample_x = x.cpu()[0,:,0]
sample_y = y.cpu()[0,:]
sample_approx = out.cpu()[0,:]

sample_x = sample_x.numpy().reshape((train_esamples))
sample_y = sample_y.numpy().reshape((train_esamples))
sample_approx = sample_approx.detach().numpy().reshape(
                                        (train_esamples))

_min = np.min(np.min(sample_y))
_max = np.max(np.max(sample_y))

plt.figure(figsize=(10,10))

plt.plot(sample_x, label='Input', c='orange')
plt.plot(sample_y, label='Ground Truth', linewidth=5.0)
plt.plot(sample_approx, 'pink', label='Approximation', linestyle='--')
plt.title('Ground Truth')
plt.legend()

plt.plot((sample_approx - sample_y)**2)
plt.title('Absolute Error')

plt.suptitle('')
plt.savefig(fig_path)
plt.tight_layout()
plt.show()

In [ ]:
device = 'cpu'
lp_loss = LpLoss(size_average=False)
#model = torch.load(PATH+'model/1_burgers_data_R10_2_800_16_256__04292021134321',map_location=torch.device('cpu'))
#choose dataset
# data_loader = 
model = torch.load('../model/fullres_burgers_data_R10_04242021095651_04242021095651', map_location=torch.device('cpu'))
"""
modes: 16
width: 1024
layers: 4
train vsamples: 500
train esamples: 1024
viscosity: 1/100

x_train shape: torch.Size([500, 1024, 2])
x_test shape: torch.Size([200, 1024, 2])
y_train shape: torch.Size([500, 1024])
y_test shape: torch.Size([200, 1024])
model parameter count: 142,621,697

"""
model.eval()
rel_err = 0.0
with torch.no_grad():
    x, y = next(iter(test_loader))
    x, y = x.to(device), y.to(device)
    out = model(x)
    rel_err = lp_loss(out.view(-1), y.view(-1)).item()
    rel_err /= batch_size
    
sample_x = x.cpu()[0,:,0]
sample_y = y.cpu()[0,:]
sample_approx = out.cpu()[0,:]

sample_x = sample_x.numpy().reshape((train_esamples))
sample_y = sample_y.numpy().reshape((train_esamples))
sample_approx = sample_approx.detach().numpy().reshape(
                                        (train_esamples))

_min = np.min(np.min(sample_y))
_max = np.max(np.max(sample_y))

plt.figure(figsize=(10,10))

plt.plot(sample_x, label='Input', c='orange')
plt.plot(sample_y, label='Ground Truth', linewidth=5.0)
plt.plot(sample_approx, 'pink', label='Approximation', linestyle='--')
plt.title('Ground Truth')
plt.legend()

plt.plot((sample_approx - sample_y)**2)
plt.title('Absolute Error')

plt.suptitle('')
plt.savefig(fig_path)
plt.tight_layout()
plt.show()